2: Load the Data

In [3]:
import pandas as pd

# Load the data
data_path = '../src/Data/rfms.csv'
df = pd.read_csv(data_path)

# Display the first few rows of the dataset
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,transaction_hour,transaction_day,transaction_month,transaction_year,TransactionStartTime_y,Recency,Frequency,Monetary,Score,Label
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,0,256,5,1,0,...,2,15,11,2018,2019-02-12 10:24:40+00:00,0,119,10.947603,1302.764769,Good
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,0,256,3,19,2,...,2,15,11,2018,2019-02-12 10:24:40+00:00,0,119,10.947603,1302.764769,Good
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,0,256,5,0,0,...,2,15,11,2018,2018-11-24 08:30:25+00:00,81,2,0.183915,0.004486,Bad
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,0,256,0,11,8,...,3,15,11,2018,2019-02-08 07:53:33+00:00,5,38,3.513670,22.253242,Good
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,0,256,3,19,2,...,3,15,11,2018,2019-02-08 07:53:33+00:00,5,38,3.513670,22.253242,Good


3: Assign Labels

In [5]:
# Establish a boundary and assign labels
def assign_labels(df, threshold=0.5):
    df['Label'] = df['Score'].apply(lambda x: 'Good' if x >= threshold else 'Bad')
    return df

# Example usage
df = assign_labels(df, threshold=0.5)
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,transaction_hour,transaction_day,transaction_month,transaction_year,TransactionStartTime_y,Recency,Frequency,Monetary,Score,Label
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,0,256,5,1,0,...,2,15,11,2018,2019-02-12 10:24:40+00:00,0,119,10.947603,1302.764769,Good
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,0,256,3,19,2,...,2,15,11,2018,2019-02-12 10:24:40+00:00,0,119,10.947603,1302.764769,Good
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,0,256,5,0,0,...,2,15,11,2018,2018-11-24 08:30:25+00:00,81,2,0.183915,0.004486,Bad
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,0,256,0,11,8,...,3,15,11,2018,2019-02-08 07:53:33+00:00,5,38,3.513670,22.253242,Good
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,0,256,3,19,2,...,3,15,11,2018,2019-02-08 07:53:33+00:00,5,38,3.513670,22.253242,Good


4: Perform WoE and IV using scorecardpy

In [25]:
import scorecardpy as sc

# Select the specified fields for WoE and IV calculation
selected_fields = ['Amount', 'Value', 'total_transaction_amount', 'average_transaction_amount', 'transaction_count', 'std_transaction_amount']

# Perform WoE Binning and calculate IV
bins = sc.woebin(df, y='Label', x=selected_fields, positive='Good')

# Convert the data to WoE values
df_woe = sc.woebin_ply(df, bins)

# Display the WoE values
df_woe.head()

# Calculate Information Value (IV) manually
def calculate_iv(bins):
    iv_values = {}
    for feature, bin_info in bins.items():
        iv = sum((bin_info['woe'] * (bin_info['good'] - bin_info['bad'])).fillna(0))
        iv_values[feature] = iv
    return iv_values

# Example usage
iv_values = calculate_iv(bins)
print(iv_values)

[INFO] creating woe binning ...


e:\OnlineClass\AIM\Bati-Bank-Credit-Score-Modelling\.venv\Lib\site-packages\scorecardpy\condition_fun.py:141: UserWarning: The positive value in "Label" was replaced by 1 and negative value by 0.
  warnings.warn("The positive value in \"{}\" was replaced by 1 and negative value by 0.".format(y))
e:\OnlineClass\AIM\Bati-Bank-Credit-Score-Modelling\.venv\Lib\site-packages\scorecardpy\condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  datetime_cols = dat.apply(pd.to_numeric,errors='ignore').select_dtypes(object).apply(pd.to_datetime,errors='ignore').select_dtypes('datetime64').columns.tolist()
e:\OnlineClass\AIM\Bati-Bank-Credit-Score-Modelling\.venv\Lib\site-packages\scorecardpy\condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instea

[INFO] converting into woe values ...
{'Amount': -10827.424028896323, 'Value': -8261.889963513784, 'total_transaction_amount': -112905.81496723673, 'average_transaction_amount': -18920.248967192125, 'transaction_count': 0.0, 'std_transaction_amount': -31548.926330906655}


5: Perform Monotonic WoE Binning using optbinning

In [26]:
from optbinning import OptimalBinning

def perform_monotonic_woe_binning(df, target='Label'):
    # Select the specified fields for WoE and IV calculation
    selected_fields = ['Amount', 'Value', 'total_transaction_amount', 'average_transaction_amount', 'transaction_count', 'std_transaction_amount']
    
    # Initialize the OptimalBinning object for each feature
    binning_results = {}
    for feature in selected_fields:
        optb = OptimalBinning(name=feature, dtype="numerical", solver="cp")
        optb.fit(df[feature], df[target])
        binning_results[feature] = optb
    
    # Transform the data using the binning results
    df_woe = df.copy()
    for feature, optb in binning_results.items():
        df_woe[feature] = optb.transform(df[feature], metric="woe")
    
    return df_woe, binning_results

# Example usage
df_woe_monotonic, binning_results = perform_monotonic_woe_binning(df, target='Label')
df_woe_monotonic.head()

(CVXPY) Jan 28 01:41:08 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 28 01:41:08 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


e:\OnlineClass\AIM\Bati-Bank-Credit-Score-Modelling\.venv\Lib\site-packages\optbinning\binning\transformations.py:38: RuntimeWarning: invalid value encountered in scalar divide
  return np.log((1. / event_rate - 1) * n_event / n_nonevent)


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,transaction_hour,transaction_day,transaction_month,transaction_year,TransactionStartTime_y,Recency,Frequency,Monetary,Score,Label
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,0,256,5,1,0,...,2,15,11,2018,2019-02-12 10:24:40+00:00,0,119,10.947603,1302.764769,Good
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,0,256,3,19,2,...,2,15,11,2018,2019-02-12 10:24:40+00:00,0,119,10.947603,1302.764769,Good
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,0,256,5,0,0,...,2,15,11,2018,2018-11-24 08:30:25+00:00,81,2,0.183915,0.004486,Bad
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,0,256,0,11,8,...,3,15,11,2018,2019-02-08 07:53:33+00:00,5,38,3.513670,22.253242,Good
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,0,256,3,19,2,...,3,15,11,2018,2019-02-08 07:53:33+00:00,5,38,3.513670,22.253242,Good


6: Calculate Information Value (IV) using optbinning

In [30]:
def calculate_iv(binning_results):
    iv_values = {}
    for feature, optb in binning_results.items():
        binning_table = optb.binning_table.build()
        iv = sum((binning_table['WoE'] * (binning_table['EventRate'] - binning_table['NonEventRate'])).fillna(0))
        iv_values[feature] = iv
    return iv_values

# Example usage
iv_values = calculate_iv(binning_results)
print(iv_values)

e:\OnlineClass\AIM\Bati-Bank-Credit-Score-Modelling\.venv\Lib\site-packages\optbinning\binning\binning_statistics.py:526: RuntimeWarning: invalid value encountered in divide
  p_nonevent = n_nonevent / t_n_nonevent
e:\OnlineClass\AIM\Bati-Bank-Credit-Score-Modelling\.venv\Lib\site-packages\optbinning\binning\binning_statistics.py:536: RuntimeWarning: divide by zero encountered in scalar divide
  constant = np.log(t_n_event / t_n_nonevent)
e:\OnlineClass\AIM\Bati-Bank-Credit-Score-Modelling\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
e:\OnlineClass\AIM\Bati-Bank-Credit-Score-Modelling\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
e:\OnlineClass\AIM\Bati-Bank-Credit-Score-Modelling\.venv\Lib\site-packages\sklearn\utils\depre

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.